In [ ]:
import pandas as pd
import networkx as nx

data = pd.read_excel(r"") # Enter your data here


def get_network_structure(data, manager_column, employee_column):
    # Create ID for unique relationships
    data['id'] = data.index + 1

    # Sort names to ignore order of relation
    data['sorted_pair'] = data.apply(lambda row: tuple(sorted([row[manager_column], row[employee_column]])), axis=1)

    # Group pairs and count occurences
    relationship_counts = data.groupby('sorted_pair').size().reset_index(name='count')

    # Create Manager and Employee column again
    relationship_counts[[manager_column, employee_column]] = pd.DataFrame(
        relationship_counts['sorted_pair'].tolist(), index=relationship_counts.index
    )

    # Create an undirected graph
    G = nx.Graph()

    # Add Edges 
    for _, row in relationship_counts.iterrows():
        G.add_edge(row[manager_column], row[employee_column], weight=row['count'])

    # Calculate positions of nodes
    positions = nx.spring_layout(G, weight='weight')

    # Calculate centricity measures
    degree_centrality = nx.degree_centrality(G)
    betweenness_centrality = nx.betweenness_centrality(G, weight='weight')
    closeness_centrality = nx.closeness_centrality(G)
    eigenvector_centrality = nx.eigenvector_centrality(G, weight='weight', max_iter=10000)


    # Create lists for node data
    nodes = []
    x_coords = []
    y_coords = []
    degrees = []
    betweennesses = []
    closenesses = []
    eigenvectors = []

    # Append values to the list
    for node in G.nodes():
        nodes.append(node)
        x_coords.append(positions[node][0])
        y_coords.append(positions[node][1])
        degrees.append(degree_centrality[node])
        betweennesses.append(betweenness_centrality[node])
        closenesses.append(closeness_centrality[node])
        eigenvectors.append(eigenvector_centrality[node])

    # Create DF for centricity meaures
    employees_df = pd.DataFrame({
        'Node': nodes,
        'X': x_coords,
        'Y': y_coords,
        'Degree Centrality': degrees,
        'Betweenness Centrality': betweennesses,
        'Closeness Centrality': closenesses,
        'Eigenvector Centrality': eigenvectors
    })

    # Create Dataframe for relationships
    Lines = relationship_counts[[manager_column, employee_column]]
    Lines['Lines'] = range(1, len(Lines) + 1)
    Lines = pd.melt(Lines, id_vars=['Lines'], value_vars=[manager_column, employee_column],
                    var_name='Role', value_name='Node')
    Lines = pd.merge(Lines, employees_df, how='left', on='Node')
    Lines = Lines[['Lines', 'Node', 'X', 'Y', 'Degree Centrality', 'Betweenness Centrality', 'Closeness Centrality', 'Eigenvector Centrality']]

    return Lines

result_df = get_network_structure(data, 'Manager', 'Employee')
result_df.to_csv('Lines.csv', index=False)  # Speichern außerhalb der Funktion